In [1]:
import pandas as pd
import re
import os
import jieba
import jieba.posseg as pseg
import pickle
import numpy as np
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
## turn back to main directory
os.chdir("../")
os.getcwd()

'/project/at082-group17'

## load mail data

In [3]:
mail = pd.read_csv('data/mail_data.csv')

In [4]:
## load 'article_cutted'
with open('data/mail_cutted', 'rb') as file:
    data = pickle.load(file)

## word count

In [5]:
counts = {}
for words in data:
    for word in words:
        if len(word) == 1:
            continue
        else:
            counts[word] = counts.get(word,0) + 1
items = list(counts.items())
items.sort(key = lambda x : x[1], reverse = True)
# for i in range(200):
#     word, count = items[i]
#     print("{0:<10}{1:>5}".format(word, count))
    


In [6]:
df = pd.DataFrame(items, columns=['word', 'count'])

In [7]:
df[df['count'] > 200]

,word,count
0,name,3558
1,優惠,2312
2,電子報,2305
3,通知,1764
4,活動,1737
5,測試,1403
6,台灣,1352
7,行銷,1307
8,科技,1244
9,課程,1217


In [8]:
## save data as pickle format
with open('data/pickle/' + 'wordCount' + '.p', "wb") as file:
    pickle.dump(df, file)

## word openRate

In [9]:
## load 'data'
dataNameList = ['201812', '201901', '201902' ,'201903']
dataCount = []
df_all = pd.DataFrame()
for name in dataNameList:
    with open('data/pickle/' + name + '.p', "rb") as file:
        loadData = pickle.load(file)
        dataCount.append(len(loadData.index))
        df_all = pd.concat((df_all, loadData), axis = 0, ignore_index = False)
print('data count ' + str(len(df_all.index)))

data count 38512321


In [10]:
df['openCount'] = 0
df['AllCount'] = 0
df.head()

,word,count,openCount,AllCount
0,name,3558,0,0
1,優惠,2312,0,0
2,電子報,2305,0,0
3,通知,1764,0,0
4,活動,1737,0,0


In [11]:
## filter rules
mail['subject'] = mail['subject'].str.replace('https?:\/\/\S*|︱', '')
mail['subject'] = mail['subject'].str.replace('【', ' ')
mail['subject'] = mail['subject'].str.replace('】', ' ')
mail['subject'] = mail['subject'].str.replace('{', ' ')
mail['subject'] = mail['subject'].str.replace('}', ' ')
mail['subject'] = mail['subject'].str.replace('$', '')
mail['subject'] = mail['subject'].str.replace('『', ' ')
mail['subject'] = mail['subject'].str.replace('』', ' ')

mail['subject'] = mail['subject'].replace('', np.nan)
mail['content'] = mail['fromName'] + ' ' + mail['subject']

In [12]:
wordList = []
for i in range(len(mail['content'])):
    wordList.append(jieba.lcut(mail['content'][i]))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.672 seconds.
Prefix dict has been built succesfully.


In [13]:
mail['subject_cut'] = wordList

In [17]:
mail_N = mail.drop(['fromName', 'fromMailAddress', 'subject', 'scheduledDate', 'content'], axis = 1)

In [25]:
mail_N['subject_cut'][0]

['server', ' ', '专属', '邀请函']

In [19]:
df_all = df_all.merge(mail_N, how="left", on="campaignSn")

In [20]:
for rowData in df_all['subject_cut']:
    words = rowData['subject_cut']
    for word in df['word']:
        if (words.count(word) == 1):
            df.loc[df['word'] == word, 'AllCount'] = df.loc[df['word'] == word, 'AllCount'] + 1
            if ()


,recipientSn,campaignSn,mailAddress,domain,openedFlag,fromMailAddress,scheduledDate,domain_count,dayofweek,time,subject_cut
0,0000000067025082016765aa02a27947,00000000676335ca016765a7eab43701,922ebdf29aa9ff88e8f23a73816ace3f938f057b2f8519...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
1,0000000067025082016765aa02a37948,00000000676335ca016765a7eab43701,dfe3cf86eb44ffe7d8a982b12908b62464c1ab81d2c6c5...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
2,0000000067025082016765aa02a37949,00000000676335ca016765a7eab43701,afd05a37e6376c7bb5db1bf0088369eb1c813b2e745660...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
3,0000000067025082016765aa02a3794a,00000000676335ca016765a7eab43701,b48b29655b5dc19908c9733ae3176d325990e011b0ca99...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
4,0000000067025082016765aa02a3794b,00000000676335ca016765a7eab43701,4ecd7ca658949e94fab23f25dd05589e6d0f30c8532ac8...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
5,0000000067025082016765aa02a3794c,00000000676335ca016765a7eab43701,08dab4b23a3f8ad688ec8cb452fa508b2e6656785347be...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
6,0000000067025082016765aa02a3794d,00000000676335ca016765a7eab43701,d0b0bd4e1ac25dfc558e50a461e2ca187b858df3bc6c5e...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
7,0000000067025082016765aa02a3794e,00000000676335ca016765a7eab43701,087e0d625d430295c640f4c093c7cd8959ef00daf734dd...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
8,0000000067025082016765aa02a3794f,00000000676335ca016765a7eab43701,72f5b4345afb34adef369b7e6aa243d7921d46c09e9697...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."
9,0000000067025082016765aa02a37950,00000000676335ca016765a7eab43701,a6597f701bc94983bb190c61bdcdb8d3c428ca2535cd6b...,gmail.com,False,chin55777@gmail.com,2018-12-01 01:28:18,4119025,5,0.926693,"[焱, 之杰, , 短片, 教, 你, 讓, , 小銅板, "", 長, "", 成大, 鈔..."


In [57]:
df.loc[df['word'] == 'name', 'AllCount'] = df.loc[df['word'] == 'name', 'AllCount'] + 1

,word,count,openCount,AllCount
0,name,3558,0,2
